In [1]:
%%capture --no-stderr
%pip install -U --quiet langchain==0.2.12 langchain-core==0.2.27 langchain-openai==0.1.20 langgraph==0.1.19 langsmith==0.1.96  numexpr langchainhub sqlalchemy langchain-community

In [3]:
import getpass
import os

from langchain_openai import ChatOpenAI

# Imported from the https://github.com/langchain-ai/langgraph/tree/main/examples/plan-and-execute repo
from tools.image_analysis_tool import get_image_analysis_tools
from tools.text2SQL import get_text2SQL_tools

from langchain_core.pydantic_v1 import BaseModel, Field
from src.joiner import *
from src.build_graph import graph_construction, graph_construction_report, graph_construction_m3ae

from langchain_core.messages import (
    BaseMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage,
)

from src.planner import *
from src.task_fetching_unit import *
from src.joiner import *
from src.joiner import parse_joiner_output
from src.utils import _get_db_schema
from typing import Dict
from src.utils import correct_malformed_json, CustomJSONEncoder
import ast
# from langgraph.checkpoint.sqlite import SqliteSaver


from langgraph.graph import END, MessageGraph, START,StateGraph

def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")

_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
# _set_if_undefined("TAVILY_API_KEY")
# # Optional, add tracing in LangSmith

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "XMODE"



In [ ]:
model="gpt-4o" #gpt-4-turbo-preview
#memory = SqliteSaver.from_conn_string(":memory:")

chain = graph_construction(model)

In [4]:

db_path="/home/ubuntu/workspace/XMODE-LLMCompiler/mimic_iv_cxr.db"
# "enumerate all detected abnormalities, given the study 57883509."
example_question="Is there any evidence in the most recent study for patient 13859433 this year indicating the continued absence of anatomical findings in the left hilar structure compared to the previous study?"
tables=['TB_CXR','PATIENTS']
tables= [t.upper()for t in tables]
# id= 2000
database_schema =_get_db_schema(db_path, tables)
#"could you list all anatomical locations related to any diseases newly detected in the 55411906 study versus the findings of the 59867439 study?"
#"how many days have elapsed from the first time patient 19631398's chest x-ray demonstrated any anatomical findings in the mediastinum in 2103?"
#"what's the count of patients whose chest x-ray studies since 15 month ago demonstrated any anatomical findings in the cardiac silhouette?"
#"did patient 12354194 undergo the procedure on two vessels procedure within the same month after a chest x-ray study revealed any anatomical findings until 2 year ago?"

inputs = {"question": example_question, "database_schema":database_schema}
# config = {"configurable": {"thread_id": "xmode-2000"}}
inputs=[HumanMessage(content=[inputs])]

for output in chain.stream(inputs, stream_mode="values"):
   print(output)
   # for key, value in output.items():
   #    print(f"output from node '{key}'")
   #    print("-----")
   #    print(value)
# _steps_dict = chain.stream(
#     [
#         HumanMessage(
#             content=[chain_input]
#         ),
    
#     ],
#     # config=config,
#     #stream_mode="debug",
# )



[HumanMessage(content=[{'question': 'is there any evidence of abnormalities still absent in the 55949143 study compared to the last study?', 'database_schema': '\nCREATE TABLE "PATIENTS" (\n\trow_id INTEGER, \n\tsubject_id INTEGER, \n\tgender TEXT, \n\tdob TEXT, \n\tdod TEXT\n)\n\n/*\n5 rows from PATIENTS table:\nrow_id\tsubject_id\tgender\tdob\tdod\n0\t10020740\tm\t2048-03-16 00:00:00\tNone\n1\t10037020\tm\t2043-02-15 00:00:00\tNone\n2\t10063856\tf\t2042-07-04 00:00:00\t2103-04-13 00:00:00\n3\t10098993\tf\t2025-02-23 00:00:00\t2105-04-17 00:00:00\n4\t10104308\tm\t2048-02-22 00:00:00\tNone\n*/\n\n\nCREATE TABLE "TB_CXR" (\n\trow_id INTEGER, \n\tsubject_id INTEGER, \n\thadm_id REAL, \n\tstudy_id INTEGER, \n\timage_id TEXT, \n\tviewposition TEXT, \n\tstudydatetime TEXT\n)\n\n/*\n5 rows from TB_CXR table:\nrow_id\tsubject_id\thadm_id\tstudy_id\timage_id\tviewposition\tstudydatetime\n0\t10020740\tNone\t52268471\t46ee8707-81386f73-3ff804c3-92e6044b-dce8d6b2\tap\t2104-03-22 16:03:20\n1\t1002

In [4]:
to_json=[]
for msg in output:
      value= msg.to_json()['kwargs']
      to_json.append(value)

In [ ]:
to_json

In [ ]:

ast.literal_eval(output[-1].content)

In [15]:
from langchain.load.dump import dumps
import json
from pprint import pprint

# print(output[-1].pretty_print())

    
# for message in output:
  
# #    message.pretty_print()
#     pprint(message)  




with open('steps_dict.json', 'w', encoding='utf-8') as f:
        json.dump(to_json, f, ensure_ascii=False, indent=4)

In [ ]:
from IPython.display import Image, display


display(Image(chain.get_graph().draw_mermaid_png()))


https://smith.langchain.com/public/c9acfa5f-c8e2-4203-bb48-9ce046621f09/r

In [ ]:
import ast
rs="{'Summary': 'No new diseases found in the right lower lung zone.', 'details': 'In the last and previous studies for patient 19103929 this year, there are no visible diseases in the right lower lung zone.', 'source': 'Image analysis of the last and previous studies.', 'inference': 'no', 'extra explanation': 'Both the last and previous studies show no diseases in the right lower lung zone.'}"
print(ast.literal_eval(rs))